In [1]:
import requests
from bs4 import BeautifulSoup

In [2]:
url = "https://results.eci.gov.in/PcResultGenJune2024/index.htm"
site = requests.get(url)
soup = BeautifulSoup(site.content,'html.parser')

In [3]:
option_html = soup.find("select", id = "ctl00_ContentPlaceHolder1_Result1_ddlState")
option_html

<select id="ctl00_ContentPlaceHolder1_Result1_ddlState" name="state" onchange="return GetResult(this)"> <option value=""> Select State Wise </option><option value="U01">Andaman &amp; Nicobar Islands</option><option value="S01">Andhra Pradesh</option><option value="S02">Arunachal Pradesh</option><option value="S03">Assam</option><option value="S04">Bihar</option><option value="U02">Chandigarh</option><option value="S26">Chhattisgarh</option><option value="U03">Dadra &amp; Nagar Haveli and Daman &amp; Diu</option><option value="S05">Goa</option><option value="S06">Gujarat</option><option value="S07">Haryana</option><option value="S08">Himachal Pradesh</option><option value="U08">Jammu and Kashmir</option><option value="S27">Jharkhand</option><option value="S10">Karnataka</option><option value="S11">Kerala</option><option value="U09">Ladakh</option><option value="U06">Lakshadweep</option><option value="S12">Madhya Pradesh</option><option value="S13">Maharashtra</option><option value="S14"

In [4]:
def finding_optionValue(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content,'html.parser')
    dropdown_result = soup.find("select", id = "ctl00_ContentPlaceHolder1_Result1_ddlState")
    options = dropdown_result.find_all('option')
    option_value_list = []
    for option in options:
        option_value = option.get('value')
        option_value_list.append(option_value)
    return option_value_list

In [5]:
states_and_ut = finding_optionValue("https://results.eci.gov.in/PcResultGenJune2024/index.htm")
states_and_ut

['',
 'U01',
 'S01',
 'S02',
 'S03',
 'S04',
 'U02',
 'S26',
 'U03',
 'S05',
 'S06',
 'S07',
 'S08',
 'U08',
 'S27',
 'S10',
 'S11',
 'U09',
 'U06',
 'S12',
 'S13',
 'S14',
 'S15',
 'S16',
 'S17',
 'U05',
 'S18',
 'U07',
 'S19',
 'S20',
 'S21',
 'S22',
 'S29',
 'S23',
 'S24',
 'S28',
 'S25']

In [6]:
 for index,element in enumerate(states_and_ut[1:]):
        url = f"https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-{element}.htm"
        page = requests.get(url)
        print(index,page.status_code)

0 200
1 200
2 200
3 200
4 200
5 200
6 200
7 200
8 200
9 200
10 200
11 200
12 200
13 200
14 200
15 200
16 200
17 200
18 200
19 200
20 200
21 200
22 200
23 200
24 200
25 200
26 200
27 200
28 200
29 200
30 200
31 200
32 200
33 200
34 200
35 200


In [7]:
master_list = []
state_code_list = []

for elements in states_and_ut[1:]:
        url = f"https://results.eci.gov.in/PcResultGenJune2024/partywiseresult-{elements}.htm"
        pc = finding_optionValue(url)
        master_list.append(pc)

In [8]:
flat_master = sum(master_list, [])

In [9]:
for elements in flat_master:
    if (elements == ""):
        flat_master.remove("")

In [10]:
len(flat_master)

543

In [11]:
import pandas as pd

In [22]:
def table_scrapper(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content,'html.parser')
    result_table = soup.find("table")
    for body in result_table.find_all('tbody'):
        rows = body.find_all('tr')
    sr_no = []
    candidate_name = []
    party_name = []
    evm_votes = []
    postal_votes = []
    total_votes = []
    percent_votes = []
    for row in rows:
        sr = row.find_all('td')[0].text
        sr_no.append(sr)
        can_name = row.find_all('td')[1].text
        candidate_name.append(can_name)
        party = row.find_all('td')[2].text
        party_name.append(party)
        e_votes = row.find_all('td')[3].text
        evm_votes.append(e_votes)
        p_votes = row.find_all('td')[4].text
        postal_votes.append(p_votes)
        t_votes = row.find_all('td')[5].text
        total_votes.append(t_votes)
        per_votes = row.find_all('td')[6].text
        percent_votes.append(per_votes)
        pc = soup.find('h2').text
        state_name = soup.find('strong').text

        data = {
        "Sr No": sr_no,
        "Candidate Name": candidate_name,
        "Party Name": party_name,
        "EVM Votes": evm_votes,
        "Postal Votes": postal_votes,
        "Total Votes": total_votes,
        "Percent Votes": percent_votes,
        "Constiteuncy" : pc,
        "State" : state_name
    }

    df = pd.DataFrame(data)
    return df

In [23]:
df_try = pd.DataFrame(columns = ["S.N.","Candidate","Party","EVM Votes","Postal Votes","Total Votes","% of Votes","Constituency"])

In [24]:
from concurrent.futures import ThreadPoolExecutor


In [25]:
def fetch_data(element):
    url = f"https://results.eci.gov.in/PcResultGenJune2024/Constituencywise{element}.htm"
    return table_scrapper(url)

# List to store the dataframes
df_list = []

# Use ThreadPoolExecutor to fetch data concurrently
with ThreadPoolExecutor(max_workers=10) as executor:
    # Fetch data concurrently and store results in a list
    results = list(executor.map(fetch_data, flat_master))

# Concatenate all dataframes at once
final_df = pd.concat(results, axis=0, ignore_index=True)

# Output the final dataframe
print(final_df)

     Sr No      Candidate Name                                   Party Name  \
0        1     BISHNU PADA RAY                       Bharatiya Janata Party   
1        2  KULDEEP RAI SHARMA                     Indian National Congress   
2        3          MANOJ PAUL          Andaman Nicobar Democratic Congress   
3        4          D AYYAPPAN          Communist Party of India  (Marxist)   
4        5     V.K. ABDUL AZIZ                                  Independent   
...    ...                 ...                                          ...   
8897     9     SK. SAPIYAR ALI                                  Independent   
8898    10         NIKHIL BERA  Socialist Unity Centre Of India (COMMUNIST)   
8899    11   AMAL KUMAR BARMAN                                  Independent   
8900    12    RAMPRASAD GHORAI                          Indian Unity Centre   
8901    13                NOTA                            None of the Above   

     EVM Votes Postal Votes Total Votes Percent Vot

In [26]:
final_df.head()

,Sr No,Candidate Name,Party Name,EVM Votes,Postal Votes,Total Votes,Percent Votes,Constiteuncy,State
0,1,BISHNU PADA RAY,Bharatiya Janata Party,102182,254,102436,50.58,Parliamentary Constituency 1 - Andaman & Nic...,(Andaman & Nicobar Islands)
1,2,KULDEEP RAI SHARMA,Indian National Congress,77829,211,78040,38.54,Parliamentary Constituency 1 - Andaman & Nic...,(Andaman & Nicobar Islands)
2,3,MANOJ PAUL,Andaman Nicobar Democratic Congress,8236,18,8254,4.08,Parliamentary Constituency 1 - Andaman & Nic...,(Andaman & Nicobar Islands)
3,4,D AYYAPPAN,Communist Party of India (Marxist),6009,8,6017,2.97,Parliamentary Constituency 1 - Andaman & Nic...,(Andaman & Nicobar Islands)
4,5,V.K. ABDUL AZIZ,Independent,2195,8,2203,1.09,Parliamentary Constituency 1 - Andaman & Nic...,(Andaman & Nicobar Islands)


In [27]:
final_df.nunique()

Sr No               55
Candidate Name    8101
Party Name         746
EVM Votes         6081
Postal Votes      1447
Total Votes       6100
Percent Votes     1557
Constiteuncy       543
State               36
dtype: int64

In [28]:
final_df['State'] = final_df['State'].str.extract(r"\((.*?)\)")
final_df

,Sr No,Candidate Name,Party Name,EVM Votes,Postal Votes,Total Votes,Percent Votes,Constiteuncy,State
0,1,BISHNU PADA RAY,Bharatiya Janata Party,102182,254,102436,50.58,Parliamentary Constituency 1 - Andaman & Nic...,Andaman & Nicobar Islands
1,2,KULDEEP RAI SHARMA,Indian National Congress,77829,211,78040,38.54,Parliamentary Constituency 1 - Andaman & Nic...,Andaman & Nicobar Islands
2,3,MANOJ PAUL,Andaman Nicobar Democratic Congress,8236,18,8254,4.08,Parliamentary Constituency 1 - Andaman & Nic...,Andaman & Nicobar Islands
3,4,D AYYAPPAN,Communist Party of India (Marxist),6009,8,6017,2.97,Parliamentary Constituency 1 - Andaman & Nic...,Andaman & Nicobar Islands
4,5,V.K. ABDUL AZIZ,Independent,2195,8,2203,1.09,Parliamentary Constituency 1 - Andaman & Nic...,Andaman & Nicobar Islands
...,...,...,...,...,...,...,...,...,...
8897,9,SK. SAPIYAR ALI,Independent,2924,-,2924,0.21,Parliamentary Constituency 26 - Uluberia (We...,West Bengal
8898,10,NIKHIL BERA,Socialist Unity Centre Of India (COMMUNIST),2095,12,2107,0.15,Parliamentary Constituency 26 - Uluberia (We...,West Bengal
8899,11,AMAL KUMAR BARMAN,Independent,1997,3,2000,0.14,Parliamentary Constituency 26 - Uluberia (We...,West Bengal
8900,12,RAMPRASAD GHORAI,Indian Unity Centre,1568,4,1572,0.11,Parliamentary Constituency 26 - Uluberia (We...,West Bengal


In [30]:
new_order = ["Sr No", "State", "Constiteuncy", "Candidate Name", "Party Name", "EVM Votes", "Postal Votes", "Total Votes", "Percent Votes"]
order_df = final_df[new_order]

In [31]:
order_df.rename(columns = {"Constiteuncy":"Constituency"},inplace = True)

In [32]:
order_df.head()

,Sr No,State,Constituency,Candidate Name,Party Name,EVM Votes,Postal Votes,Total Votes,Percent Votes
0,1,Andaman & Nicobar Islands,Parliamentary Constituency 1 - Andaman & Nic...,BISHNU PADA RAY,Bharatiya Janata Party,102182,254,102436,50.58
1,2,Andaman & Nicobar Islands,Parliamentary Constituency 1 - Andaman & Nic...,KULDEEP RAI SHARMA,Indian National Congress,77829,211,78040,38.54
2,3,Andaman & Nicobar Islands,Parliamentary Constituency 1 - Andaman & Nic...,MANOJ PAUL,Andaman Nicobar Democratic Congress,8236,18,8254,4.08
3,4,Andaman & Nicobar Islands,Parliamentary Constituency 1 - Andaman & Nic...,D AYYAPPAN,Communist Party of India (Marxist),6009,8,6017,2.97
4,5,Andaman & Nicobar Islands,Parliamentary Constituency 1 - Andaman & Nic...,V.K. ABDUL AZIZ,Independent,2195,8,2203,1.09


In [34]:
order_df.to_csv("final_election_result.csv",index = False)